<a href="https://colab.research.google.com/github/lcbjrrr/quantai/blob/main/J_N_N.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
versioninfo()

# Neural Networks (with Flux)

In [1]:
import Pkg
Pkg.add("CSV")
Pkg.add("DataFrames")
using CSV
using DataFrames

    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.10/Project.toml`
  No Changes to `~/.julia/environments/v1.10/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.10/Project.toml`
  No Changes to `~/.julia/environments/v1.10/Manifest.toml`


In [2]:
;wget https://raw.githubusercontent.com/lcbjrrr/data/main/banking%20-%20train.csv

--2025-03-15 01:01:40--  https://raw.githubusercontent.com/lcbjrrr/data/main/banking%20-%20train.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 222 [text/plain]
Saving to: ‘banking - train.csv’

     0K                                                       100% 10.0M=0s

2025-03-15 01:01:40 (10.0 MB/s) - ‘banking - train.csv’ saved [222/222]



In [3]:
;wget https://raw.githubusercontent.com/lcbjrrr/data/main/banking%20-%20tests.csv

--2025-03-15 01:01:40--  https://raw.githubusercontent.com/lcbjrrr/data/main/banking%20-%20tests.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 170 [text/plain]
Saving to: ‘banking - tests.csv’

     0K                                                       100% 8.47M=0s

2025-03-15 01:01:40 (8.47 MB/s) - ‘banking - tests.csv’ saved [170/170]



In [4]:
Pkg.add("Flux")
Pkg.add("StatsBase")
Pkg.add("EvalMetrics")
using Flux
using StatsBase
using EvalMetrics

   Resolving package versions...
   Installed ZygoteRules ────────── v0.2.7
   Installed ContextVariablesX ──── v0.1.3
   Installed InitialValues ──────── v0.3.1
   Installed ShowCases ──────────── v0.1.0
   Installed ScopedValues ───────── v1.3.0
   Installed Accessors ──────────── v0.1.42
   Installed MLUtils ────────────── v0.4.7
   Installed GPUArrays ──────────── v11.2.2
   Installed PrettyPrint ────────── v0.2.0
   Installed FLoopsBase ─────────── v0.1.1
   Installed ProgressLogging ────── v0.1.4
   Installed RealDot ────────────── v0.1.0
   Installed MLCore ─────────────── v1.0.0
   Installed OneHotArrays ───────── v0.2.6
   Installed IRTools ────────────── v0.4.14
   Installed Zygote ─────────────── v0.7.4
   Installed MicroCollections ───── v0.2.0
   Installed DefineSingletons ───── v0.1.2
   Installed NameResolution ─────── v0.1.5
   Installed BangBang ───────────── v0.4.4
   Installed FLoops ─────────────── v0.2.2
   Installed ChainRules ─────────── v1.72.3
   Installed MLSt

In [5]:
train=CSV.read("banking - train.csv", DataFrame)
first(train,5)

Row,salary,balance,default
,Float64,Float64,Int64
1,1000.0,100.0,0
2,950.0,95.0,0
3,800.0,85.0,0
4,1100.0,5500.0,0
5,999.0,6000.0,0


In [6]:
test=CSV.read("banking - tests.csv", DataFrame)
first(test,5)

Row,salary,balance,default
,Float64,Float64,Int64
1,1050.0,105.0,1
2,997.5,99.75,0
3,1048.95,6300.0,0
4,932.4,6825.0,0
5,2415.0,4200.0,0


In [7]:
X_train  = transpose(hcat([ row.salary for row in eachrow(train)],[ row.balance for row in eachrow(train)]))

2×16 transpose(::Matrix{Float64}) with eltype Float64:
 1000.0  950.0  800.0  1100.0   999.0   888.0  2300.0  …  5000.0  3333.0  4444.0  5555.0  6666.66
  100.0   95.0   85.0  5500.0  6000.0  6500.0  4000.0       85.0  5500.0  6000.0  6500.0  6666.66

In [8]:
y_train = transpose([row.default for row in eachrow(train)])

1×16 transpose(::Vector{Int64}) with eltype Int64:
 0  0  0  0  0  0  0  0  0  1  1  1  0  0  0  0

In [27]:
EP = 20
N=length(y_train)
BS=floor(Int,N/EP)
data = Flux.DataLoader((X_train, y_train), batchsize=BS,shuffle=true);
nn    = Chain(
  Dense(2,4, relu),
  Dense(4,3, relu),
  Dense(3,1,identity),
  softmax
  )
loss(x, y) = Flux.Losses.binarycrossentropy(Float64.(nn(x)), Float32.(y))
adam=ADAM()
for i in 1:EP
    # train (loss func, Flux.params(nn), data/batches, optimizer)
    Flux.train!(loss,nn,data,adam)
end
pred_train  =  transpose([nn(X_train[i:i+1])[1] for i in 1:2:N*2])
pred_train


┌ Warning: Optimiser Adam has state which will be discarded after `train!` finishes.
│ Please run `opt = Flux.setup(Adam(), model)` and pass this `opt` to `train!`.
│   leaf = Leaf(Adam(eta=0.001, beta=(0.9, 0.999), epsilon=1.0e-8), (Float32[0.0 0.0; 0.0 0.0; 0.0 0.0; 0.0 0.0], Float32[0.0 0.0; 0.0 0.0; 0.0 0.0; 0.0 0.0], (0.9, 0.999)))
└ @ Flux.Train ~/.julia/packages/Flux/3711C/src/train.jl:140


LoadError: MethodError: no method matching loss(::Chain{Tuple{Dense{typeof(relu), Matrix{Float32}, Vector{Float32}}, Dense{typeof(relu), Matrix{Float32}, Vector{Float32}}, Dense{typeof(identity), Matrix{Float32}, Vector{Float32}}, typeof(softmax)}}, ::Vector{Float64}, ::Vector{Int64})

[0mClosest candidates are:
[0m  loss(::Any, ::Any)
[0m[90m   @[39m [35mMain[39m [90m[4mIn[27]:11[24m[39m


In [10]:
function accuracy(y,pred)
  acc = sum(pred.==y)/length(y)
  return acc
end

accuracy(y_train, pred_train)*100

LoadError: UndefVarError: `pred_train` not defined

In [11]:
X_test = transpose(hcat([ row.salary for row in eachrow(test)],[ row.balance for row in eachrow(test)]))
y_test = transpose([row.default for row in eachrow(test)])
print(y_test)



[1 0 0 0 0 0 0 1 0 1]

In [12]:
pred_test  =  transpose([ Int64(nn(X_test[i:i+1])[1]) for i in 1:2:length(y_test)*2])
print(pred_test)

LoadError: UndefVarError: `nn` not defined

In [13]:
accuracy(y_test, pred_test)

LoadError: UndefVarError: `pred_test` not defined

In [14]:
counts( y_test,pred_test)


LoadError: UndefVarError: `pred_test` not defined

In [15]:
precision(vec(y_test),vec(pred_test))

LoadError: UndefVarError: `pred_test` not defined

In [16]:
recall(vec(y_test),vec(pred_test)) )

LoadError: UndefVarError: `pred_test` not defined